In [1]:
import polars as pl
from IPython.display import display
import json
import ast

In [2]:
encode_df = pl.read_parquet('../encode_files.parquet')

In [3]:
for column in encode_df.iter_columns():
    display(column.unique())

title
str
"""ENCFF819LHK"""
"""ENCFF978MSE"""
"""ENCFF111MFR"""
"""ENCFF822KRV"""
"""ENCFF039ZBL"""
…
"""ENCFF514KLY"""
"""ENCFF332FTS"""
"""ENCFF594BFE"""


accession
str
"""ENCFF606MHR"""
"""ENCFF545OLF"""
"""ENCFF020IMS"""
"""ENCFF822EAJ"""
"""ENCFF323ZIV"""
…
"""ENCFF615GWD"""
"""ENCFF633CMO"""
"""ENCFF997MRC"""


dataset
str
"""/experiments/ENCSR271VMW/"""
"""/experiments/ENCSR411AFQ/"""
"""/experiments/ENCSR847RRR/"""
"""/experiments/ENCSR807KQN/"""
"""/experiments/ENCSR812VCN/"""
…
"""/experiments/ENCSR392YGP/"""
"""/experiments/ENCSR475ZYS/"""
"""/experiments/ENCSR558SEE/"""


assembly
str
"""ce10"""
"""dm6"""
"""dm3"""
"""mm10-minimal"""
"""GRCh38-minimal"""
…
"""NA"""
"""hg19"""
"""mm10"""


technical_replicates
str
"""['1_99']"""
"""['1_2', '2_2']"""
"""['1_16']"""
"""['2_3']"""
"""['1_1', '1_2', '2_1', '2_2', '…"
…
"""['1_1', '1_2', '1_3', '1_4', '…"
"""['5_1', '5_2']"""
"""['2_3', '2_4']"""


...

status
str
"""released"""
"""archived"""


annotation_type
str
"""NA"""


annotation_subtype
str
"""NA"""


biochemical_inputs
str
"""NA"""


encyclopedia_version
str
"""NA"""


In [4]:
display(encode_df)

title,accession,dataset,assembly,technical_replicates,biological_replicates,file_format,file_type,file_format_type,file_size,assay_term_name,term_name,organ_slims,simple_biosample_summary,origin_batches,label,download_url,derived_from,genome_annotation,paired_end,paired_with,preferred_default,run_type,read_length,mapped_read_length,cropped_read_length,cropped_read_length_tolerance,mapped_run_type,read_length_units,output_category,output_type,index_of,quality_metrics,lab_title,project,step_run,date_created,analysis_step_version,restricted,submitter_comment,status,annotation_type,annotation_subtype,biochemical_inputs,encyclopedia_version
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""ENCFF566NGG""","""ENCFF566NGG""","""/experiments/ENCSR369MDF/""","""GRCh38""","""['1_1']""","""[1]""","""bam""","""bam""","""NA""","""3936091129""","""RNA-seq""","""K562""","""['blood', 'bodily fluid']""","""treated with 5 μM JQ1 for 4 ho…","""['/biosamples/ENCBS291NHT/']""","""NA""","""https://encodeproject.org/file…","""['/files/ENCFF598IDH/', '/file…","""V29""","""NA""","""NA""","""NA""","""NA""","""NA""","""150""","""NA""","""NA""","""paired-ended""","""nt""","""alignment""","""transcriptome alignments""","""NA""","""[{'assay_term_name': 'RNA-seq'…","""ENCODE Processing Pipeline""","""ENCODE""","""{'status': 'released', 'lab': …","""2021-12-31T08:51:07.433754+00:…","""{'software_versions': [{'submi…","""NA""","""NA""","""released""","""NA""","""NA""","""NA""","""NA"""
"""ENCFF642UFR""","""ENCFF642UFR""","""/experiments/ENCSR250SCW/""","""mm10""","""['2_1']""","""[2]""","""tar""","""tar""","""NA""","""4117922""","""single-cell RNA sequencing ass…","""left cerebral cortex""","""['brain']""","""male adult (2 months) strain B…","""NA""","""NA""","""https://encodeproject.org/file…","""['/files/ENCFF723SJO/', '/file…","""M21""","""NA""","""NA""","""True""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""quantification""","""unfiltered sparse gene count m…","""NA""","""[{'assay_term_name': 'single-c…","""Barbara Wold, Caltech""","""ENCODE""","""{'status': 'released', 'date_c…","""2022-01-30T04:31:33.614985+00:…","""{'date_created': '2021-12-14T2…","""NA""","""NA""","""released""","""NA""","""NA""","""NA""","""NA"""
"""ENCFF094MID""","""ENCFF094MID""","""/experiments/ENCSR414JJE/""","""NA""","""['1_2']""","""[1]""","""fastq""","""fastq""","""NA""","""263567745""","""Mint-ChIP-seq""","""naive B cell""","""['blood', 'bodily fluid']""","""female adult (39 years)""","""['/biosamples/ENCBS517ZRS/']""","""H3K27me3""","""https://encodeproject.org/file…","""NA""","""NA""","""2""","""/files/ENCFF481AOK/""","""NA""","""paired-ended""","""58""","""NA""","""NA""","""NA""","""NA""","""nt""","""raw data""","""reads""","""NA""","""[]""","""Bradley Bernstein, Broad""","""ENCODE""","""NA""","""2022-02-11T14:58:17.561304+00:…","""NA""","""NA""","""NA""","""released""","""NA""","""NA""","""NA""","""NA"""
"""ENCFF506TJN""","""ENCFF506TJN""","""/experiments/ENCSR369MDF/""","""GRCh38""","""['2_1']""","""[2]""","""bigWig""","""bigWig""","""NA""","""120998658""","""RNA-seq""","""K562""","""['blood', 'bodily fluid']""","""treated with 5 μM JQ1 for 4 ho…","""['/biosamples/ENCBS798JQV/']""","""NA""","""https://encodeproject.org/file…","""['/files/ENCFF124OPS/', '/file…","""V29""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""signal""","""signal of all reads""","""NA""","""[]""","""ENCODE Processing Pipeline""","""ENCODE""","""{'status': 'released', 'lab': …","""2021-12-31T08:51:14.572055+00:…","""{'analysis_step': {'documents'…","""NA""","""NA""","""released""","""NA""","""NA""","""NA""","""NA"""
"""ENCFF174IGS""","""ENCFF174IGS""","""/experiments/ENCSR040TXN/""","""NA""","""['1_2']""","""[1]""","""fastq""","""fastq""","""NA""","""45801960""","""Mint-ChIP-seq""","""central memory CD

In [5]:
# Convert "NA" to None

encode_df = pl.DataFrame([column.replace({"NA": None}) for column in encode_df.iter_columns()])
display(encode_df.head())

title,accession,dataset,assembly,technical_replicates,biological_replicates,file_format,file_type,file_format_type,file_size,assay_term_name,term_name,organ_slims,simple_biosample_summary,origin_batches,label,download_url,derived_from,genome_annotation,paired_end,paired_with,preferred_default,run_type,read_length,mapped_read_length,cropped_read_length,cropped_read_length_tolerance,mapped_run_type,read_length_units,output_category,output_type,index_of,quality_metrics,lab_title,project,step_run,date_created,analysis_step_version,restricted,submitter_comment,status,annotation_type,annotation_subtype,biochemical_inputs,encyclopedia_version
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""ENCFF566NGG""","""ENCFF566NGG""","""/experiments/ENCSR369MDF/""","""GRCh38""","""['1_1']""","""[1]""","""bam""","""bam""",null,"""3936091129""","""RNA-seq""","""K562""","""['blood', 'bodily fluid']""","""treated with 5 μM JQ1 for 4 ho…","""['/biosamples/ENCBS291NHT/']""",null,"""https://encodeproject.org/file…","""['/files/ENCFF598IDH/', '/file…","""V29""",null,null,null,null,null,"""150""",null,null,"""paired-ended""","""nt""","""alignment""","""transcriptome alignments""",null,"""[{'assay_term_name': 'RNA-seq'…","""ENCODE Processing Pipeline""","""ENCODE""","""{'status': 'released', 'lab': …","""2021-12-31T08:51:07.433754+00:…","""{'software_versions': [{'submi…",null,null,"""released""",null,null,null,null
"""ENCFF642UFR""","""ENCFF642UFR""","""/experiments/ENCSR250SCW/""","""mm10""","""['2_1']""","""[2]""","""tar""","""tar""",null,"""4117922""","""single-cell RNA sequencing ass…","""left cerebral cortex""","""['brain']""","""male adult (2 months) strain B…",null,null,"""https://encodeproject.org/file…","""['/files/ENCFF723SJO/', '/file…","""M21""",null,null,"""True""",null,null,null,null,null,null,null,"""quantification""","""unfiltered sparse gene count m…",null,"""[{'assay_term_name': 'single-c…","""Barbara Wold, Caltech""","""ENCODE""","""{'status': 'released', 'date_c…","""2022-01-30T04:31:33.614985+00:…","""{'date_created': '2021-12-14T2…",null,null,"""released""",null,null,null,null
"""ENCFF094MID""","""ENCFF094MID""","""/experiments/ENCSR414JJE/""",null,"""['1_2']""","""[1]""","""fastq""","""fastq""",null,"""263567745""","""Mint-ChIP-seq""","""naive B cell""","""['blood', 'bodily fluid']""","""female adult (39 years)""","""['/biosamples/ENCBS517ZRS/']""","""H3K27me3""","""https://encodeproject.org/file…",null,null,"""2""","""/files/ENCFF481AOK/""",null,"""paired-ended""","""58""",null,null,null,null,"""nt""","""raw data""","""reads""",null,"""[]""","""Bradley Bernstein, Broad""","""ENCODE""",null,"""2022-02-11T14:58:17.561304+00:…",null,null,null,"""released""",null,null,null,null
"""ENCFF506TJN""","""ENCFF506TJN""","""/experiments/ENCSR369MDF/""","""GRCh38""","""['2_1']""","""[2]""","""bigWig""","""bigWig""",null,"""120998658""","""RNA-seq""","""K562""","""['blood', 'bodily fluid']""","""treated with 5 μM JQ1 for 4 ho…","""['/biosamples/ENCBS798JQV/']""",null,"""https://encodeproject.org/file…","""['/files/ENCFF124OPS/', '/file…","""V29""",null,null,null,null,null,null,null,null,null,null,"""signal""","""signal of all reads""",null,"""[]""","""ENCODE Processing Pipeline""","""ENCODE""","""{'status': 'released', 'lab': …","""2021-12-31T08:51:14.572055+00:…","""{'analysis_step': {'documents'…",null,null,"""released""",null,null,null,null
"""ENCFF174IGS""","""ENCFF174IGS""","""/experiments/ENCSR040TXN/""",null,"""['1_2']""","""[1]""","""fastq""","""fastq""",null,"""45801960""","""Mint-ChIP-seq""","""central memory CD8-positive, a…","""[]""","""male adult (36 years)""","""['/biosamples/ENCBS181QGR/']""","""H3K4me3""","""https://encodeproject.org/file…",null,null,"""2""","""/files/ENCFF066EBL/""",null,"""paired-ended""","""58""",null,null,null,null,"""nt""","""raw data""","""reads""",null,"""[]""","""Bradley 

In [6]:
# Drop last 4 columns (contain no information)

encode_df = encode_df.drop(pl.col("annotation_type", "annotation_subtype", "biochemical_inputs", "encyclopedia_version"))
display(encode_df.head())

title,accession,dataset,assembly,technical_replicates,biological_replicates,file_format,file_type,file_format_type,file_size,assay_term_name,term_name,organ_slims,simple_biosample_summary,origin_batches,label,download_url,derived_from,genome_annotation,paired_end,paired_with,preferred_default,run_type,read_length,mapped_read_length,cropped_read_length,cropped_read_length_tolerance,mapped_run_type,read_length_units,output_category,output_type,index_of,quality_metrics,lab_title,project,step_run,date_created,analysis_step_version,restricted,submitter_comment,status
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""ENCFF566NGG""","""ENCFF566NGG""","""/experiments/ENCSR369MDF/""","""GRCh38""","""['1_1']""","""[1]""","""bam""","""bam""",null,"""3936091129""","""RNA-seq""","""K562""","""['blood', 'bodily fluid']""","""treated with 5 μM JQ1 for 4 ho…","""['/biosamples/ENCBS291NHT/']""",null,"""https://encodeproject.org/file…","""['/files/ENCFF598IDH/', '/file…","""V29""",null,null,null,null,null,"""150""",null,null,"""paired-ended""","""nt""","""alignment""","""transcriptome alignments""",null,"""[{'assay_term_name': 'RNA-seq'…","""ENCODE Processing Pipeline""","""ENCODE""","""{'status': 'released', 'lab': …","""2021-12-31T08:51:07.433754+00:…","""{'software_versions': [{'submi…",null,null,"""released"""
"""ENCFF642UFR""","""ENCFF642UFR""","""/experiments/ENCSR250SCW/""","""mm10""","""['2_1']""","""[2]""","""tar""","""tar""",null,"""4117922""","""single-cell RNA sequencing ass…","""left cerebral cortex""","""['brain']""","""male adult (2 months) strain B…",null,null,"""https://encodeproject.org/file…","""['/files/ENCFF723SJO/', '/file…","""M21""",null,null,"""True""",null,null,null,null,null,null,null,"""quantification""","""unfiltered sparse gene count m…",null,"""[{'assay_term_name': 'single-c…","""Barbara Wold, Caltech""","""ENCODE""","""{'status': 'released', 'date_c…","""2022-01-30T04:31:33.614985+00:…","""{'date_created': '2021-12-14T2…",null,null,"""released"""
"""ENCFF094MID""","""ENCFF094MID""","""/experiments/ENCSR414JJE/""",null,"""['1_2']""","""[1]""","""fastq""","""fastq""",null,"""263567745""","""Mint-ChIP-seq""","""naive B cell""","""['blood', 'bodily fluid']""","""female adult (39 years)""","""['/biosamples/ENCBS517ZRS/']""","""H3K27me3""","""https://encodeproject.org/file…",null,null,"""2""","""/files/ENCFF481AOK/""",null,"""paired-ended""","""58""",null,null,null,null,"""nt""","""raw data""","""reads""",null,"""[]""","""Bradley Bernstein, Broad""","""ENCODE""",null,"""2022-02-11T14:58:17.561304+00:…",null,null,null,"""released"""
"""ENCFF506TJN""","""ENCFF506TJN""","""/experiments/ENCSR369MDF/""","""GRCh38""","""['2_1']""","""[2]""","""bigWig""","""bigWig""",null,"""120998658""","""RNA-seq""","""K562""","""['blood', 'bodily fluid']""","""treated with 5 μM JQ1 for 4 ho…","""['/biosamples/ENCBS798JQV/']""",null,"""https://encodeproject.org/file…","""['/files/ENCFF124OPS/', '/file…","""V29""",null,null,null,null,null,null,null,null,null,null,"""signal""","""signal of all reads""",null,"""[]""","""ENCODE Processing Pipeline""","""ENCODE""","""{'status': 'released', 'lab': …","""2021-12-31T08:51:14.572055+00:…","""{'analysis_step': {'documents'…",null,null,"""released"""
"""ENCFF174IGS""","""ENCFF174IGS""","""/experiments/ENCSR040TXN/""",null,"""['1_2']""","""[1]""","""fastq""","""fastq""",null,"""45801960""","""Mint-ChIP-seq""","""central memory CD8-positive, a…","""[]""","""male adult (36 years)""","""['/biosamples/ENCBS181QGR/']""","""H3K4me3""","""https://encodeproject.org/file…",null,null,"""2""","""/files/ENCFF066EBL/""",null,"""paired-ended""","""58""",null,null,null,null,"""nt""","""raw data""","""reads""",null,"""[]""","""Bradley Bernstein, Broad""","""ENCODE""",null,"""2022-02-11T15:54:31.696914+00:…",null,null,null,"""released"""


In [7]:
# Convert select columns to numeric


encode_df = encode_df.with_columns(pl.col("file_size", "read_length", "mapped_read_length", "cropped_read_length", "cropped_read_length_tolerance").cast(pl.Float64))
display(encode_df)

title,accession,dataset,assembly,technical_replicates,biological_replicates,file_format,file_type,file_format_type,file_size,assay_term_name,term_name,organ_slims,simple_biosample_summary,origin_batches,label,download_url,derived_from,genome_annotation,paired_end,paired_with,preferred_default,run_type,read_length,mapped_read_length,cropped_read_length,cropped_read_length_tolerance,mapped_run_type,read_length_units,output_category,output_type,index_of,quality_metrics,lab_title,project,step_run,date_created,analysis_step_version,restricted,submitter_comment,status
str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""ENCFF566NGG""","""ENCFF566NGG""","""/experiments/ENCSR369MDF/""","""GRCh38""","""['1_1']""","""[1]""","""bam""","""bam""",null,3.9361e9,"""RNA-seq""","""K562""","""['blood', 'bodily fluid']""","""treated with 5 μM JQ1 for 4 ho…","""['/biosamples/ENCBS291NHT/']""",null,"""https://encodeproject.org/file…","""['/files/ENCFF598IDH/', '/file…","""V29""",null,null,null,null,null,150.0,null,null,"""paired-ended""","""nt""","""alignment""","""transcriptome alignments""",null,"""[{'assay_term_name': 'RNA-seq'…","""ENCODE Processing Pipeline""","""ENCODE""","""{'status': 'released', 'lab': …","""2021-12-31T08:51:07.433754+00:…","""{'software_versions': [{'submi…",null,null,"""released"""
"""ENCFF642UFR""","""ENCFF642UFR""","""/experiments/ENCSR250SCW/""","""mm10""","""['2_1']""","""[2]""","""tar""","""tar""",null,4.117922e6,"""single-cell RNA sequencing ass…","""left cerebral cortex""","""['brain']""","""male adult (2 months) strain B…",null,null,"""https://encodeproject.org/file…","""['/files/ENCFF723SJO/', '/file…","""M21""",null,null,"""True""",null,null,null,null,null,null,null,"""quantification""","""unfiltered sparse gene count m…",null,"""[{'assay_term_name': 'single-c…","""Barbara Wold, Caltech""","""ENCODE""","""{'status': 'released', 'date_c…","""2022-01-30T04:31:33.614985+00:…","""{'date_created': '2021-12-14T2…",null,null,"""released"""
"""ENCFF094MID""","""ENCFF094MID""","""/experiments/ENCSR414JJE/""",null,"""['1_2']""","""[1]""","""fastq""","""fastq""",null,2.63567745e8,"""Mint-ChIP-seq""","""naive B cell""","""['blood', 'bodily fluid']""","""female adult (39 years)""","""['/biosamples/ENCBS517ZRS/']""","""H3K27me3""","""https://encodeproject.org/file…",null,null,"""2""","""/files/ENCFF481AOK/""",null,"""paired-ended""",58.0,null,null,null,null,"""nt""","""raw data""","""reads""",null,"""[]""","""Bradley Bernstein, Broad""","""ENCODE""",null,"""2022-02-11T14:58:17.561304+00:…",null,null,null,"""released"""
"""ENCFF506TJN""","""ENCFF506TJN""","""/experiments/ENCSR369MDF/""","""GRCh38""","""['2_1']""","""[2]""","""bigWig""","""bigWig""",null,1.20998658e8,"""RNA-seq""","""K562""","""['blood', 'bodily fluid']""","""treated with 5 μM JQ1 for 4 ho…","""['/biosamples/ENCBS798JQV/']""",null,"""https://encodeproject.org/file…","""['/files/ENCFF124OPS/', '/file…","""V29""",null,null,null,null,null,null,null,null,null,null,"""signal""","""signal of all reads""",null,"""[]""","""ENCODE Processing Pipeline""","""ENCODE""","""{'status': 'released', 'lab': …","""2021-12-31T08:51:14.572055+00:…","""{'analysis_step': {'documents'…",null,null,"""released"""
"""ENCFF174IGS""","""ENCFF174IGS""","""/experiments/ENCSR040TXN/""",null,"""['1_2']""","""[1]""","""fastq""","""fastq""",null,4.580196e7,"""Mint-ChIP-seq""","""central memory CD8-positive, a…","""[]""","""male adult (36 years)""","""['/biosamples/ENCBS181QGR/']""","""H3K4me3""","""https://encodeproject.org/file…",null,null,"""2""","""/files/ENCFF066EBL/""",null,"""paired-ended""",58.0,null,null,null,null,"""nt""","""raw data""","""reads""",null,"""[]""","""Bradley Bernstein, Broad""","""ENCODE""",null,"""2022-02-11T15:54:31.696914+00:…",null,null,null,"""released"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ENCFF266RHM""",

In [8]:
# Convert select columns to boolean

encode_df = encode_df.with_columns(
    pl.col("preferred_default", "restricted").map_elements(
        lambda x: True if x.lower() in ["true"] else 
                  False if x.lower() in ["false"] else None, return_dtype=pl.Boolean)
)
display(encode_df)

title,accession,dataset,assembly,technical_replicates,biological_replicates,file_format,file_type,file_format_type,file_size,assay_term_name,term_name,organ_slims,simple_biosample_summary,origin_batches,label,download_url,derived_from,genome_annotation,paired_end,paired_with,preferred_default,run_type,read_length,mapped_read_length,cropped_read_length,cropped_read_length_tolerance,mapped_run_type,read_length_units,output_category,output_type,index_of,quality_metrics,lab_title,project,step_run,date_created,analysis_step_version,restricted,submitter_comment,status
str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,bool,str,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,bool,str,str
"""ENCFF566NGG""","""ENCFF566NGG""","""/experiments/ENCSR369MDF/""","""GRCh38""","""['1_1']""","""[1]""","""bam""","""bam""",null,3.9361e9,"""RNA-seq""","""K562""","""['blood', 'bodily fluid']""","""treated with 5 μM JQ1 for 4 ho…","""['/biosamples/ENCBS291NHT/']""",null,"""https://encodeproject.org/file…","""['/files/ENCFF598IDH/', '/file…","""V29""",null,null,null,null,null,150.0,null,null,"""paired-ended""","""nt""","""alignment""","""transcriptome alignments""",null,"""[{'assay_term_name': 'RNA-seq'…","""ENCODE Processing Pipeline""","""ENCODE""","""{'status': 'released', 'lab': …","""2021-12-31T08:51:07.433754+00:…","""{'software_versions': [{'submi…",null,null,"""released"""
"""ENCFF642UFR""","""ENCFF642UFR""","""/experiments/ENCSR250SCW/""","""mm10""","""['2_1']""","""[2]""","""tar""","""tar""",null,4.117922e6,"""single-cell RNA sequencing ass…","""left cerebral cortex""","""['brain']""","""male adult (2 months) strain B…",null,null,"""https://encodeproject.org/file…","""['/files/ENCFF723SJO/', '/file…","""M21""",null,null,true,null,null,null,null,null,null,null,"""quantification""","""unfiltered sparse gene count m…",null,"""[{'assay_term_name': 'single-c…","""Barbara Wold, Caltech""","""ENCODE""","""{'status': 'released', 'date_c…","""2022-01-30T04:31:33.614985+00:…","""{'date_created': '2021-12-14T2…",null,null,"""released"""
"""ENCFF094MID""","""ENCFF094MID""","""/experiments/ENCSR414JJE/""",null,"""['1_2']""","""[1]""","""fastq""","""fastq""",null,2.63567745e8,"""Mint-ChIP-seq""","""naive B cell""","""['blood', 'bodily fluid']""","""female adult (39 years)""","""['/biosamples/ENCBS517ZRS/']""","""H3K27me3""","""https://encodeproject.org/file…",null,null,"""2""","""/files/ENCFF481AOK/""",null,"""paired-ended""",58.0,null,null,null,null,"""nt""","""raw data""","""reads""",null,"""[]""","""Bradley Bernstein, Broad""","""ENCODE""",null,"""2022-02-11T14:58:17.561304+00:…",null,null,null,"""released"""
"""ENCFF506TJN""","""ENCFF506TJN""","""/experiments/ENCSR369MDF/""","""GRCh38""","""['2_1']""","""[2]""","""bigWig""","""bigWig""",null,1.20998658e8,"""RNA-seq""","""K562""","""['blood', 'bodily fluid']""","""treated with 5 μM JQ1 for 4 ho…","""['/biosamples/ENCBS798JQV/']""",null,"""https://encodeproject.org/file…","""['/files/ENCFF124OPS/', '/file…","""V29""",null,null,null,null,null,null,null,null,null,null,"""signal""","""signal of all reads""",null,"""[]""","""ENCODE Processing Pipeline""","""ENCODE""","""{'status': 'released', 'lab': …","""2021-12-31T08:51:14.572055+00:…","""{'analysis_step': {'documents'…",null,null,"""released"""
"""ENCFF174IGS""","""ENCFF174IGS""","""/experiments/ENCSR040TXN/""",null,"""['1_2']""","""[1]""","""fastq""","""fastq""",null,4.580196e7,"""Mint-ChIP-seq""","""central memory CD8-positive, a…","""[]""","""male adult (36 years)""","""['/biosamples/ENCBS181QGR/']""","""H3K4me3""","""https://encodeproject.org/file…",null,null,"""2""","""/files/ENCFF066EBL/""",null,"""paired-ended""",58.0,null,null,null,null,"""nt""","""raw data""","""reads""",null,"""[]""","""Bradley Bernstein, Broad""","""ENCODE""",null,"""2022-02-11T15:54:31.696914+00:…",null,null,null,"""released"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ENCFF266RHM""","""E

In [9]:
# Convert select columns to datetime

encode_df = encode_df.with_columns(pl.col("date_created").cast(pl.Datetime))
display(encode_df)


title,accession,dataset,assembly,technical_replicates,biological_replicates,file_format,file_type,file_format_type,file_size,assay_term_name,term_name,organ_slims,simple_biosample_summary,origin_batches,label,download_url,derived_from,genome_annotation,paired_end,paired_with,preferred_default,run_type,read_length,mapped_read_length,cropped_read_length,cropped_read_length_tolerance,mapped_run_type,read_length_units,output_category,output_type,index_of,quality_metrics,lab_title,project,step_run,date_created,analysis_step_version,restricted,submitter_comment,status
str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,bool,str,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,datetime[μs],str,bool,str,str
"""ENCFF566NGG""","""ENCFF566NGG""","""/experiments/ENCSR369MDF/""","""GRCh38""","""['1_1']""","""[1]""","""bam""","""bam""",null,3.9361e9,"""RNA-seq""","""K562""","""['blood', 'bodily fluid']""","""treated with 5 μM JQ1 for 4 ho…","""['/biosamples/ENCBS291NHT/']""",null,"""https://encodeproject.org/file…","""['/files/ENCFF598IDH/', '/file…","""V29""",null,null,null,null,null,150.0,null,null,"""paired-ended""","""nt""","""alignment""","""transcriptome alignments""",null,"""[{'assay_term_name': 'RNA-seq'…","""ENCODE Processing Pipeline""","""ENCODE""","""{'status': 'released', 'lab': …",2021-12-31 08:51:07.433754,"""{'software_versions': [{'submi…",null,null,"""released"""
"""ENCFF642UFR""","""ENCFF642UFR""","""/experiments/ENCSR250SCW/""","""mm10""","""['2_1']""","""[2]""","""tar""","""tar""",null,4.117922e6,"""single-cell RNA sequencing ass…","""left cerebral cortex""","""['brain']""","""male adult (2 months) strain B…",null,null,"""https://encodeproject.org/file…","""['/files/ENCFF723SJO/', '/file…","""M21""",null,null,true,null,null,null,null,null,null,null,"""quantification""","""unfiltered sparse gene count m…",null,"""[{'assay_term_name': 'single-c…","""Barbara Wold, Caltech""","""ENCODE""","""{'status': 'released', 'date_c…",2022-01-30 04:31:33.614985,"""{'date_created': '2021-12-14T2…",null,null,"""released"""
"""ENCFF094MID""","""ENCFF094MID""","""/experiments/ENCSR414JJE/""",null,"""['1_2']""","""[1]""","""fastq""","""fastq""",null,2.63567745e8,"""Mint-ChIP-seq""","""naive B cell""","""['blood', 'bodily fluid']""","""female adult (39 years)""","""['/biosamples/ENCBS517ZRS/']""","""H3K27me3""","""https://encodeproject.org/file…",null,null,"""2""","""/files/ENCFF481AOK/""",null,"""paired-ended""",58.0,null,null,null,null,"""nt""","""raw data""","""reads""",null,"""[]""","""Bradley Bernstein, Broad""","""ENCODE""",null,2022-02-11 14:58:17.561304,null,null,null,"""released"""
"""ENCFF506TJN""","""ENCFF506TJN""","""/experiments/ENCSR369MDF/""","""GRCh38""","""['2_1']""","""[2]""","""bigWig""","""bigWig""",null,1.20998658e8,"""RNA-seq""","""K562""","""['blood', 'bodily fluid']""","""treated with 5 μM JQ1 for 4 ho…","""['/biosamples/ENCBS798JQV/']""",null,"""https://encodeproject.org/file…","""['/files/ENCFF124OPS/', '/file…","""V29""",null,null,null,null,null,null,null,null,null,null,"""signal""","""signal of all reads""",null,"""[]""","""ENCODE Processing Pipeline""","""ENCODE""","""{'status': 'released', 'lab': …",2021-12-31 08:51:14.572055,"""{'analysis_step': {'documents'…",null,null,"""released"""
"""ENCFF174IGS""","""ENCFF174IGS""","""/experiments/ENCSR040TXN/""",null,"""['1_2']""","""[1]""","""fastq""","""fastq""",null,4.580196e7,"""Mint-ChIP-seq""","""central memory CD8-positive, a…","""[]""","""male adult (36 years)""","""['/biosamples/ENCBS181QGR/']""","""H3K4me3""","""https://encodeproject.org/file…",null,null,"""2""","""/files/ENCFF066EBL/""",null,"""paired-ended""",58.0,null,null,null,null,"""nt""","""raw data""","""reads""",null,"""[]""","""Bradley Bernstein, Broad""","""ENCODE""",null,2022-02-11 15:54:31.696914,null,null,null,"""released"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ENCFF266RHM""","""ENCFF266RHM""","""/experiments/ENCSR8

In [10]:
# Test ast.literal_eval

example = "[]"
result = ast.literal_eval(example)
print(result)
print(type(result))

[]
<class 'list'>


In [11]:
# Convert select columns to valid JSON

encode_df = encode_df.with_columns(pl.col("technical_replicates", "biological_replicates", "organ_slims", "origin_batches", "derived_from").map_elements(lambda x: json.loads(x.replace("'", '"'))))
display(encode_df)

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredict

title,accession,dataset,assembly,technical_replicates,biological_replicates,file_format,file_type,file_format_type,file_size,assay_term_name,term_name,organ_slims,simple_biosample_summary,origin_batches,label,download_url,derived_from,genome_annotation,paired_end,paired_with,preferred_default,run_type,read_length,mapped_read_length,cropped_read_length,cropped_read_length_tolerance,mapped_run_type,read_length_units,output_category,output_type,index_of,quality_metrics,lab_title,project,step_run,date_created,analysis_step_version,restricted,submitter_comment,status
str,str,str,str,list[str],list[i64],str,str,str,f64,str,str,list[str],str,list[str],str,str,list[str],str,str,str,bool,str,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,datetime[μs],str,bool,str,str
"""ENCFF566NGG""","""ENCFF566NGG""","""/experiments/ENCSR369MDF/""","""GRCh38""","[""1_1""]",[1],"""bam""","""bam""",null,3.9361e9,"""RNA-seq""","""K562""","[""blood"", ""bodily fluid""]","""treated with 5 μM JQ1 for 4 ho…","[""/biosamples/ENCBS291NHT/""]",null,"""https://encodeproject.org/file…","[""/files/ENCFF598IDH/"", ""/files/ENCFF146IQN/"", ""/files/ENCFF967MFO/""]","""V29""",null,null,null,null,null,150.0,null,null,"""paired-ended""","""nt""","""alignment""","""transcriptome alignments""",null,"""[{'assay_term_name': 'RNA-seq'…","""ENCODE Processing Pipeline""","""ENCODE""","""{'status': 'released', 'lab': …",2021-12-31 08:51:07.433754,"""{'software_versions': [{'submi…",null,null,"""released"""
"""ENCFF642UFR""","""ENCFF642UFR""","""/experiments/ENCSR250SCW/""","""mm10""","[""2_1""]",[2],"""tar""","""tar""",null,4.117922e6,"""single-cell RNA sequencing ass…","""left cerebral cortex""","[""brain""]","""male adult (2 months) strain B…",null,null,"""https://encodeproject.org/file…","[""/files/ENCFF723SJO/"", ""/files/ENCFF192SYW/""]","""M21""",null,null,true,null,null,null,null,null,null,null,"""quantification""","""unfiltered sparse gene count m…",null,"""[{'assay_term_name': 'single-c…","""Barbara Wold, Caltech""","""ENCODE""","""{'status': 'released', 'date_c…",2022-01-30 04:31:33.614985,"""{'date_created': '2021-12-14T2…",null,null,"""released"""
"""ENCFF094MID""","""ENCFF094MID""","""/experiments/ENCSR414JJE/""",null,"[""1_2""]",[1],"""fastq""","""fastq""",null,2.63567745e8,"""Mint-ChIP-seq""","""naive B cell""","[""blood"", ""bodily fluid""]","""female adult (39 years)""","[""/biosamples/ENCBS517ZRS/""]","""H3K27me3""","""https://encodeproject.org/file…",null,null,"""2""","""/files/ENCFF481AOK/""",null,"""paired-ended""",58.0,null,null,null,null,"""nt""","""raw data""","""reads""",null,"""[]""","""Bradley Bernstein, Broad""","""ENCODE""",null,2022-02-11 14:58:17.561304,null,null,null,"""released"""
"""ENCFF506TJN""","""ENCFF506TJN""","""/experiments/ENCSR369MDF/""","""GRCh38""","[""2_1""]",[2],"""bigWig""","""bigWig""",null,1.20998658e8,"""RNA-seq""","""K562""","[""blood"", ""bodily fluid""]","""treated with 5 μM JQ1 for 4 ho…","[""/biosamples/ENCBS798JQV/""]",null,"""https://encodeproject.org/file…","[""/files/ENCFF124OPS/"", ""/files/GRCh38_EBV.chrom.sizes/""]","""V29""",null,null,null,null,null,null,null,null,null,null,"""signal""","""signal of all reads""",null,"""[]""","""ENCODE Processing Pipeline""","""ENCODE""","""{'status': 'released', 'lab': …",2021-12-31 08:51:14.572055,"""{'analysis_step': {'documents'…",null,null,"""released"""
"""ENCFF174IGS""","""ENCFF174IGS""","""/experiments/ENCSR040TXN/""",null,"[""1_2""]",[1],"""fastq""","""fastq""",null,4.580196e7,"""Mint-ChIP-seq""","""central memory CD8-positive, a…",[],"""male adult (36 years)""","[""/biosamples/ENCBS181QGR/""]","""H3K4me3""","""https://encodeproject.org/file…",null,null,"""2""","""/files/ENCFF066EBL/""",null,"""paired-ended""",58.0,null,null,null,null,"""nt""","""raw data""","""reads""",null,"""[]""","""Bradley Bernstein, Broad""","""ENCODE""",null,2022-02-11 15:54:31.696914,null,null,null,"""released"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…

In [12]:
# Inspect data

for column in encode_df["quality_metrics", "step_run", "analysis_step_version"].iter_columns():
    display(column.to_list())

["[{'assay_term_name': 'RNA-seq', 'status': 'released', 'lab': '/labs/encode-processing-pipeline/', 'award': '/awards/U24HG009397/', 'aliases': [], 'date_created': '2021-12-31T08:51:22.402334+00:00', 'submitted_by': '/users/6bae687f-b77a-46b9-af0e-a02c135cf42e/', 'schema_version': '8', 'step_run': '/analysis-step-runs/0ef6bdca-0a26-4f44-a59c-7d6edddf871e/', 'quality_metric_of': ['/files/ENCFF566NGG/'], 'attachment': {'download': 'rep1ENCSR369MDF_anno.bam.json', 'href': '@@download/attachment/rep1ENCSR369MDF_anno.bam.json', 'type': 'application/json', 'md5sum': '43baa9222d21578ad7d99663f603ec04'}, 'diff_chroms': 0, 'diff_chroms_qc_failed': 0, 'duplicates': 0, 'duplicates_qc_failed': 0, 'mapped': 84938342, 'mapped_pct': '97.2', 'mapped_qc_failed': 0, 'paired': 28108292, 'paired_properly': 25661984, 'paired_properly_pct': '91.3', 'paired_properly_qc_failed': 0, 'paired_qc_failed': 0, 'read1': 14054146, 'read1_qc_failed': 0, 'read2': 14054146, 'read2_qc_failed': 0, 'singletons': 0, 'single

["{'status': 'released', 'lab': '/labs/encode-processing-pipeline/', 'award': '/awards/U24HG009397/', 'date_created': '2021-12-31T08:51:05.489151+00:00', 'submitted_by': '/users/6bae687f-b77a-46b9-af0e-a02c135cf42e/', 'aliases': ['encode-processing-pipeline:_analysis-steps_bulk-rna-seq-alignment-step-v-1_-a7457a0d-a627-4df1-80ba-1c8de2a41eba-bdcac4249199303e3742c3635e3874df40dc11943f1809ec23f17f07a0b81ce9'], 'schema_version': '5', 'analysis_step_version': '/analysis-step-versions/bulk-rna-seq-alignment-step-v-1-0/', '@id': '/analysis-step-runs/0ef6bdca-0a26-4f44-a59c-7d6edddf871e/', '@type': ['AnalysisStepRun', 'Item'], 'uuid': '0ef6bdca-0a26-4f44-a59c-7d6edddf871e'}",
 "{'status': 'released', 'date_created': '2021-12-14T22:16:46.875646+00:00', 'submitted_by': '/users/85c75ed3-269b-4273-b4ba-e269ee350dd6/', 'aliases': ['barbara-wold:starsolo-split-quantification-step-run'], 'schema_version': '5', 'analysis_step_version': '/analysis-step-versions/starsolo-quantification-step-v-1-1/', '@

["{'software_versions': [{'submitted_by': '/users/7e944358-625c-426c-975c-7d875e22f753/', 'version': '1.9', 'download_checksum': 'e2f2e401af7e8309075ba4545118819d', 'aliases': ['encode:samtools 1.9'], 'status': 'released', 'schema_version': '4', 'software': {'lab': '/labs/encode-consortium/', 'award': '/awards/ENCODE/', 'references': [], 'date_created': '2015-02-12T21:54:17.918472+00:00', 'submitted_by': '/users/81a6cc12-2847-4e2e-8f2c-f566699eb29e/', 'url': 'http://www.htslib.org/', 'status': 'released', 'aliases': [], 'schema_version': '10', 'software_type': ['other'], 'name': 'samtools', 'title': 'Samtools', 'source_url': 'http://sourceforge.net/projects/samtools/files/samtools/', 'used_by': ['community', 'ENCODE'], 'description': 'Samtools is a suite of programs for interacting with high-throughput sequencing data.  SAMtools implements various utilities for post-processing alignments in the SAM format, such as indexing, variant caller and alignment viewer, and thus provides univers

In [ ]:
# Convert select columns to valid JSON

encode_df = encode_df.select(pl.col("quality_metrics", "step_run", "analysis_step_version").map_elements(lambda x: ast.literal_eval(x)))
display(encode_df)

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredict

In [ ]:
# Done

clean_encode = encode_df

In [ ]:
display(clean_encode)